# Task
In this tutorial, you will learn how to use if_expression.

There is a special package needed to import, which is random.

In [1]:
import random

from dflow import (OutputArtifact, OutputParameter, Outputs, Step, Steps,
            Workflow, if_expression)
from dflow.python import (OP, OPIO, Artifact, OPIOSign, PythonOPTemplate,
                    upload_packages)

if "__file__" in locals():
    upload_packages.append(__file__)

**Random OP**
This OP's outputs include five parameters, as shown below.
- `is_header` will get a bool type value through random function, true or false.
- `msg1` is "head",
- `msg2` is "tail",
- `foo` is txt format with content of "head"
- `bar` includes content of "tail"


In [2]:
class Random(OP):
    @classmethod
    def get_input_sign(cls):
        return OPIOSign()

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            "is_head": bool,
            "msg1": str,
            "msg2": str,
            "foo": Artifact(str),
            "bar": Artifact(str)
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        open("foo.txt", "w").write("head")
        open("bar.txt", "w").write("tail")
        if random.random() < 0.5:
            is_head = True
        else:
            is_head = False
        return OPIO({
            "is_head": is_head,
            "msg1": "head",
            "msg2": "tail",
            "foo": "foo.txt",
            "bar": "bar.txt"
        })

Believe you remember what you have learned from `dflow-reuse.py` about `Steps`. There, `Steps` has a output parameter and artifact.

The Random defined above is used here, using python3.8

In [ ]:
steps = Steps(
    name="conditional-steps", 
    outputs=Outputs(
    parameters={"msg": OutputParameter()},
    artifacts={"res": OutputArtifact()})
)

random_step = Step(
    name="random",
    template=PythonOPTemplate(Random, image="python:3.8")
)

steps.add(random_step)

# if_expression

In this part, `if_expression` is used to determine which message and artifact will be used. To illustrate: 
- if `is_head` is true, then `steps.outputs.parameters`'s `msg` will be `msg1` and `res` will be `foo` that includes the content of head.
- else, `steps.outputs.parameters`'s `msg` will be `msg2` and `res` will be `bar` that includes the content of tail.

In [ ]:
steps.outputs.parameters["msg"].value_from_expression = if_expression(
    _if=random_step.outputs.parameters["is_head"],
    _then=random_step.outputs.parameters["msg1"],
    _else=random_step.outputs.parameters["msg2"])

steps.outputs.artifacts["res"].from_expression = if_expression(
    _if=random_step.outputs.parameters["is_head"],
    _then=random_step.outputs.artifacts["foo"],
    _else=random_step.outputs.artifacts["bar"])

In [ ]:
wf = Workflow(name="conditional", steps=steps)
wf.submit();

Workflow has been submitted (ID: conditional-jdcfh)
